# Equation de Laplace

Considérons le problème de Laplace canonique suivant : nous cherchons une fonction $u:[0,1]\to\mathbb{R}$ qui vérifie le problème
\begin{equation}
\left\lbrace
\begin{aligned}
&-\Delta u(x) = f(x),&&0<x<1,\\
&u(0)=u(1)=0,
\end{aligned}
\right.
\end{equation}
où $f:[0,1]\to\mathbb{R}$ est une fonction régulière (par exemple continue).

Ce problème peut se résoudre directement. La solution est donnée par la formule
\begin{equation}
u(x) = x \int_0^1 \int_0^t f(s) \, \operatorname{d}\!s \, \operatorname{d}\!t - \int_0^x \int_0^t f(s) \, \operatorname{d}\!s \, \operatorname{d}\!t .
\end{equation}

La méthode des différences finies consiste à construire un maillage de l'intervalle $[0,1]$, c'est-à-dire une famille de points $x_i = \frac{i}{N+1}$, $0\leq i\leq N+1$, pour $N\in\mathbb{N}$. Le pas d'espace $\Delta x$ est défini comme la distance entre les points du maillage : $\Delta x = \frac{1}{N+1}$.
Puis nous définissons $u_i$, $0\leq i\leq N+1$, comme l'approximation de $u(x_i)$ par la formule
\begin{equation}
-\frac{u_{i+1}-2u_i+u_{i-1}}{\Delta x^2} = f_i, \qquad 1\leq i\leq N,
\end{equation}
où $f_i=f(x_i)$, $1\leq i\leq N$.
Evidemment nous complétons ces relations à l'aide des conditions de bord : $u_0=u_{N+1}=0$.

Nous pouvons écrire l'ensemble de ces relations sous une forme matricielle. Nous définissons $U = (u_1,\ldots,u_N)^t \in\mathbb{R}^N$, $F = (f_1,\ldots,f_N)^t\in\mathbb{R}^N$ et $A$ la matrice de taille $N{\times}N$ par
$$ 
A = \frac{1}{\Delta x^2}\begin{pmatrix}
2&-1&0&\ldots&0\\
-1&2&-1&\ddots&\vdots\\
0&\ddots&\ddots&\ddots&0\\
\vdots&\ddots&-1&2&-1\\
0&\ldots&0&-1&2
\end{pmatrix}.
$$
Nous pouvons donc récrire le problème sous forme matricielle
$$ Au = F.$$

In [1]:
#include <iostream>
#include <vector>
#include <stdexcept>

#include "xplot/xfigure.hpp"
#include "xplot/xmarks.hpp"
#include "xplot/xaxes.hpp"

#include "mesh1D.hpp"
#include "solution1D.hpp"
#include "matrix.hpp"
#include "lu.hpp"

### Question 1

Créez un objet de type `parameters` contenant les informations suivantes
* `xmin` l'abscisse du bord gauche ($0$ dans notre cas)
* `xmax` l'abscisse du bord droit ($1$ dans notre cas)
* `Nx` le nombre de points du maillage (à votre convenance mais ne soyez pas trop gourmand dans un premier temps)
* `node-centered` qui doit prendre la valeur $1$ de manière à construire un maillage centré sur les noeuds (et pas sur les cellules)

### Question 2

Créez une `lambda`-fonction pour le second membre de l'équation. Ici nous pourrons commencer avec la fonction $f(x) = 1$, $\forall x\in[0,1]$.

In [2]:
// parameters declaration
parameters p = {
    {"xmin", 0.},         // left bound
    {"xmax", 1.},         // right bound
    {"Nx", 32},           // number of points
    {"node-centered", 1.} // choice of the mesh
};
// right member of the Laplace equation
auto f = [](double x) { return 1.; };

### Question 3

Dans le cas où $f(x)=1$, la solution évidente du problème s'écrit
$$ u(x) = \frac{1}{2} x(1-x).$$

Créez une fonction qui prend en argument $x$ un vecteur de `double` et qui retourne $y$ un vecteur de même taille tel que 
$$ y_k = u(x_k) .$$

In [3]:
// solution
auto sol(std::vector<double> x)
{
    std::vector<double> y(x.size());
    std::transform(x.begin(), x.end(), y.begin(), [](double xk){return .5*xk*(1-xk); });
    return y;
}

### Question 4

* Fabriquez tous les éléments du problème : 
    * la solution `S` (objet de type `solution1D`) à partir des paramètres,
    * la matrice $A$ du Laplacien dont la taille est $N{\times}N$ avec $N=N_x-2$ où $N_x$ est le nombre de points du maillage comprenant les deux points de bord, 
    * le second membre $F$ qui doit être un `vector` de taille $N$ également ;
* résolvez le système linéaire $Au=F$ en utilisant la décomposition $LU$ de la matrice $A$. Le vecteur `u_in` obtenu est un vecteur de taille $N$ ;
* copiez les valeurs de `u_in` à leur place dans le vecteur `S.u` qui est de taille $N_x=N+2$.

In [4]:
solution1D S(p);

auto N = S.m.Nx - 2;
auto A = 2.*identity(N,N) - identity(N,N,1) - identity(N,N,-1);
A *= (N+1)*(N+1);
std::vector<double> F(N);
std::transform(std::next(S.m.x.begin()),
               std::prev(S.m.x.end()),
               F.begin(), f);
auto u_in = solve_lu(lu(A), F);
std::copy(u_in.begin(), u_in.end(), std::next(S.u.begin()));

### Question 5

Affichez le résultat obtenu et comparez le avec la solution exacte.
Essayez d'obtenir un graphique semblable à celui-ci.

<img src="img/TP02_Laplacien.png", width='75%'>

In [5]:
xpl::linear_scale sx, sy;
sx.min = p["xmin"], sx.max = p["xmax"];
sy.min = 0., sy.max = .125;

// fine mesh for the exact solution
auto NN = 1025;
double dx = 1./(NN-1);
std::vector<double> xx(NN);
for (auto i=0; i<NN; ++i)
    xx[i] = i*dx;

auto ax_x = xpl::axis_generator(sx)
    .label("x")
    .finalize();
auto ax_y = xpl::axis_generator(sy)
    .orientation("vertical")
    .side("left")
    .finalize();
auto line = xpl::lines_generator(sx, sy)
    .x(xx)
    .y(sol(xx))
    .colors(std::vector<std::string>({"orange"}))
    .labels(std::vector<std::string>({"exact"}))
    .display_legend(true)
    .finalize();
auto scatter = xpl::scatter_generator(sx, sy)
    .x(S.m.x)
    .y(S.u)
    .colors(std::vector<xtl::xoptional<std::string>>{"navy"})
    .labels(std::vector<std::string>({"numeric"}))
    .display_legend(true)
    .marker("circle")
    .finalize();
auto fig = xpl::figure_generator()
    .padding_x(0.025)
    .padding_y(0.1)
    .title("solution with N = " + std::to_string(S.m.Nx))
    .legend_location("top-left")
    .finalize();

fig.add_mark(line);
fig.add_mark(scatter);
fig.add_axis(ax_x);
fig.add_axis(ax_y);
fig.display();

A Jupyter widget